In [8]:
import time
import os
import cfbd
from cfbd.models.betting_game import BettingGame
from cfbd.models.season_type import SeasonType
from cfbd.rest import ApiException
from pprint import pprint
import pandas as pd
from datetime import datetime

def get_betlines(year=2025, season_type="regular", home='georgia tech', away='clemson'):   
    # Defining the host is optional and defaults to https://api.collegefootballdata.com
    # See configuration.py for a list of all supported configuration parameters.
    configuration = cfbd.Configuration(
        host = "https://api.collegefootballdata.com"
    )

    # The client must configure the authentication and authorization parameters
    # in accordance with the API server security policy.
    # Examples for each auth method are provided below, use the example that
    # satisfies your auth use case.

    # Configure Bearer authorization: apiKey
    configuration = cfbd.Configuration(
        access_token = os.environ["BEARER_TOKEN"]
    )

    # Enter a context with an instance of the API client
    with cfbd.ApiClient(configuration) as api_client:
        # Create an instance of the API class
        api_instance = cfbd.BettingApi(api_client)
        
        ##boilerplate 
        # game_id = 56 # int | Optional gameId filter (optional)
        # year = 2025 # int | Year filter, required if game id not specified (optional)
        # season_type = "regular" # SeasonType | Optional season type filter (optional)
        # home = 'georgia tech' # str | Optional home team filter (optional)
        # away = 'clemson' # str | Optional away team filter (optional)
        # conference = 'conference_example' # str | Optional conference filter (optional)
        # provider = 'provider_example' # str | Optional provider name filter (optional)

        try:
            api_response = api_instance.get_lines(year=year, season_type=season_type, home=home, away=away) # this is the json file with all the gotten data in int. 
            print("The response of BettingApi->get_lines:\n")
            pprint(api_response)
        except Exception as e:
            print("Exception when calling BettingApi->get_lines: %s\n" % e)

        return api_response

In [ ]:
import math

def run_betlines_for_duration(duration_hours=3.5, sleep_time = 15, filename="betlines_full.csv"):
    start_time = time.time()
    all_dfs = []
    iterations = math.ceil((duration_hours * 60) / sleep_time)
    
    for _ in range(iterations):
        print(f"Iteration #{_}")
        api_response = get_betlines()
        df = pd.DataFrame([bet.to_dict() for bet in api_response])
        df['time_stamp'] = current_datetime = datetime.now()
        all_dfs.append(df)
        
        # Save each iteration as before (optional)
        # timestamp = datetime.now().strftime("%Y%m%d_%H%M")
        # filename = f"betlines_{timestamp}.csv"
        # df.to_csv(filename, index=False)
        # print(f"Saved {filename}")
        
        # Wait for 15 minutes unless time is up
        if (time.time() - start_time) >= duration_hours * 3600: 
            break
        time.sleep(sleep_time * 60)
    
    # Concatenate all DataFrames and save to a big CSV
    big_df = pd.concat(all_dfs, ignore_index=True)
    big_filename = "betlines_full.csv"
    big_df.to_csv(big_filename, index=False)
    print(f"Saved all data to {big_filename}")

In [ ]:
run_betlines_for_duration(duration_hours=(1/30), sleep_time = 1)

Iteration #0
The response of BettingApi->get_lines:

[BettingGame(id=401754623, season=2025, season_type=<SeasonType.REGULAR: 'regular'>, week=3, start_date=datetime.datetime(2025, 9, 13, 16, 0, tzinfo=datetime.timezone.utc), home_team_id=59, home_team='Georgia Tech', home_conference='ACC', home_classification=<DivisionClassification.FBS: 'fbs'>, home_score=24, away_team_id=228, away_team='Clemson', away_conference='ACC', away_classification=<DivisionClassification.FBS: 'fbs'>, away_score=21, lines=[GameLine(provider='ESPN Bet', spread=2.5, formatted_spread='Clemson -2.5', spread_open=9.5, over_under=49.5, over_under_open=58.5, home_moneyline=130, away_moneyline=-150), GameLine(provider='DraftKings', spread=3, formatted_spread='Clemson -3', spread_open=8.5, over_under=51.5, over_under_open=None, home_moneyline=120, away_moneyline=-142), GameLine(provider='Bovada', spread=3, formatted_spread='Clemson -3.0', spread_open=6.5, over_under=51, over_under_open=52.5, home_moneyline=175, away_m